In [7]:
# DETECTOR LAYOUTPARSER INTELIGENTE - VERSIÓN SIMPLIFICADA
import re
import fitz  # PyMuPDF
import numpy as np
from PIL import Image
import cv2
import os
from dataclasses import dataclass
from typing import List

# Importar LayoutParser
try:
    import layoutparser as lp
    LAYOUTPARSER_AVAILABLE = True
    print("✅ LayoutParser disponible")
except ImportError:
    LAYOUTPARSER_AVAILABLE = False
    print("❌ LayoutParser no disponible")

# CONFIGURACIÓN CENTRALIZADA
RENDER_DPI = 200
DEBUG_OUTPUT_DIR = "../data/debug"
CROP_TOP = 150
CROP_BOTTOM = 140
CROP_LEFT = 50
CROP_RIGHT = 50

os.makedirs(DEBUG_OUTPUT_DIR, exist_ok=True)

@dataclass
class ElementoLayout:
    tipo: str
    bbox: tuple
    confianza: float
    texto: str = ""
    
print(f"🚀 Configuración lista - Debug: {DEBUG_OUTPUT_DIR}")

✅ LayoutParser disponible
🚀 Configuración lista - Debug: ../data/debug


In [8]:
# DETECTOR INTELIGENTE LAYOUTPARSER
class SmartLayoutDetector:
    """Detector inteligente usando LayoutParser sin dependencias externas"""
    
    def __init__(self):
        self.label_map = {"text": "red", "title": "blue", "list": "green", "table": "purple", "figure": "pink"}
        print("📦 SmartLayoutDetector inicializado")
    
    def detect(self, image):
        """Detecta elementos usando análisis multiescala"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        h, w = gray.shape
        elements = lp.Layout()
        
        # Múltiples thresholds más permisivos
        thresholds = [150, 180, 200, 220, 240]
        all_contours = []
        
        for thresh_val in thresholds:
            _, thresh = cv2.threshold(gray, thresh_val, 255, cv2.THRESH_BINARY_INV)
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            print(f"    Threshold {thresh_val}: {len(contours)} contornos")
            all_contours.extend(contours)
        
        # Procesar contornos
        processed_boxes = []
        
        print(f"    📊 Total contornos encontrados: {len(all_contours)}")
        
        rejected_small = 0
        rejected_duplicate = 0
        
        for contour in all_contours:
            x, y, cw, ch = cv2.boundingRect(contour)
            area = cw * ch
            
            # Filtros más permisivos para debug
            if cw < 20 or ch < 8 or area < 200:
                rejected_small += 1
                continue
            
            # Evitar duplicados
            is_duplicate = False
            for existing_box in processed_boxes:
                if self._has_overlap((x, y, x+cw, y+ch), existing_box['bbox']):
                    is_duplicate = True
                    rejected_duplicate += 1
                    break
            
            if not is_duplicate:
                block_type, confidence = self._classify_block(x, y, cw, ch, w, h)
                processed_boxes.append({
                    'bbox': (x, y, x+cw, y+ch),
                    'type': block_type,
                    'confidence': confidence
                })
        
        print(f"    🔍 Rechazados por tamaño: {rejected_small}")
        print(f"    🔍 Rechazados por duplicado: {rejected_duplicate}")
        print(f"    ✅ Bloques válidos procesados: {len(processed_boxes)}")
        
        # Crear elementos LayoutParser
        for box in processed_boxes:
            x1, y1, x2, y2 = box['bbox']
            element = lp.TextBlock(
                block=lp.Rectangle(x1, y1, x2, y2),
                type=box['type'],
                score=box['confidence']
            )
            elements.append(element)
        
        return elements
    
    def _has_overlap(self, box1, box2):
        """Verifica si dos cajas se solapan significativamente"""
        x1_max = max(box1[0], box2[0])
        y1_max = max(box1[1], box2[1])
        x2_min = min(box1[2], box2[2])
        y2_min = min(box1[3], box2[3])
        
        if x2_min <= x1_max or y2_min <= y1_max:
            return False
        
        intersection = (x2_min - x1_max) * (y2_min - y1_max)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        return (intersection / area1) > 0.7
    
    def _classify_block(self, x, y, w, h, img_w, img_h):
        """Clasifica bloque por características geométricas"""
        aspect_ratio = w / h
        rel_width = w / img_w
        
        if h > 40 and rel_width > 0.6:
            return "title", 0.85
        elif aspect_ratio > 10 and h < 20:
            return "figure", 0.75
        elif rel_width < 0.3 and aspect_ratio < 2:
            return "list", 0.7
        elif w > 300 and h > 100:
            return "table", 0.8
        else:
            return "text", 0.75

# Crear detector
detector = SmartLayoutDetector()
print("✅ Detector inteligente creado")

📦 SmartLayoutDetector inicializado
✅ Detector inteligente creado


In [9]:
# FUNCIONES DE PROCESAMIENTO
def page_to_image(page):
    """Convierte página PDF a imagen OpenCV"""
    scale = RENDER_DPI / 72.0
    mat = fitz.Matrix(scale, scale)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    pil_img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
    cv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    return cv_img, scale

def crop_margins(image):
    """Recorta márgenes de la imagen"""
    h, w = image.shape[:2]
    left = max(0, CROP_LEFT)
    top = max(0, CROP_TOP)
    right = min(w, w - CROP_RIGHT)
    bottom = min(h, h - CROP_BOTTOM)
    
    if left >= right or top >= bottom:
        print("⚠️ Recorte inválido, usando imagen completa")
        return image, 0, 0
    
    cropped = image[top:bottom, left:right]
    print(f"📐 Recorte: {w}x{h} → {cropped.shape[1]}x{cropped.shape[0]}")
    return cropped, left, top

def create_visualization(cv_image, layout, save_path):
    """Crea visualización usando método simple compatible"""
    if not layout or len(layout) == 0:
        print(f"  ⚠️ Layout vacío, creando imagen base")
        base_path = save_path.replace('.png', '')
        orig_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        Image.fromarray(orig_rgb).save(f"{base_path}_basic.png")
        Image.fromarray(orig_rgb).save(f"{base_path}_detailed.png")
        return cv_image, cv_image
    
    try:
        # Método manual para evitar problemas de getsize
        viz_image = cv_image.copy()
        
        colors = {
            'text': (0, 0, 255),     # Rojo en BGR
            'title': (255, 0, 0),    # Azul en BGR
            'list': (0, 255, 0),     # Verde en BGR
            'table': (128, 0, 128),  # Púrpura en BGR
            'figure': (203, 192, 255) # Rosa en BGR
        }
        
        for i, element in enumerate(layout):
            x1, y1, x2, y2 = int(element.block.x_1), int(element.block.y_1), int(element.block.x_2), int(element.block.y_2)
            color = colors.get(element.type, (255, 255, 255))
            
            # Dibujar rectángulo
            cv2.rectangle(viz_image, (x1, y1), (x2, y2), color, 3)
            
            # Añadir etiqueta
            score = getattr(element, 'score', 0.0)
            label = f"{element.type}/{score:.2f}"
            cv2.putText(viz_image, label, (x1+5, y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Número del elemento
            cv2.putText(viz_image, str(i+1), (x2-30, y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Guardar versiones
        base_path = save_path.replace('.png', '')
        
        basic_rgb = cv2.cvtColor(viz_image, cv2.COLOR_BGR2RGB)
        detailed_rgb = basic_rgb.copy()  # Misma imagen por simplicidad
        
        Image.fromarray(basic_rgb).save(f"{base_path}_basic.png")
        Image.fromarray(detailed_rgb).save(f"{base_path}_detailed.png")
        
        print(f"  💾 Básica: {base_path}_basic.png")
        print(f"  💾 Detallada: {base_path}_detailed.png")
        
        return viz_image, viz_image
        
    except Exception as e:
        print(f"  ❌ Error en visualización: {e}")
        # Fallback: imagen original
        base_path = save_path.replace('.png', '')
        orig_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        Image.fromarray(orig_rgb).save(f"{base_path}_basic.png")
        return cv_image, cv_image

def extract_text_from_layout(pdf_page, layout, scale, offset_x=0, offset_y=0):
    """Extrae texto de elementos del layout"""
    elementos_con_texto = []
    
    for element in layout:
        x1, y1, x2, y2 = element.block.x_1, element.block.y_1, element.block.x_2, element.block.y_2
        
        # Ajustar coordenadas
        adj_x1 = (x1 + offset_x) / scale
        adj_y1 = (y1 + offset_y) / scale
        adj_x2 = (x2 + offset_x) / scale
        adj_y2 = (y2 + offset_y) / scale
        
        # Extraer texto
        rect = fitz.Rect(adj_x1, adj_y1, adj_x2, adj_y2)
        text = pdf_page.get_textbox(rect).strip()
        text = re.sub(r'\s+', ' ', text)
        
        if text and len(text.strip()) > 3:
            elemento = ElementoLayout(
                tipo=element.type,
                bbox=(x1, y1, x2, y2),
                confianza=getattr(element, 'score', 0.0),
                texto=text
            )
            elementos_con_texto.append(elemento)
    
    return elementos_con_texto

print("✅ Funciones de procesamiento listas")

✅ Funciones de procesamiento listas


In [10]:
# ANÁLISIS PRINCIPAL SIMPLIFICADO
def analyze_pdf_simple(pdf_path):
    """Análisis simple y robusto"""
    print(f"🎯 ANÁLISIS LAYOUTPARSER: {os.path.basename(pdf_path)}")
    print("=" * 60)
    
    try:
        # Abrir PDF
        doc = fitz.open(pdf_path)
        page = doc[0]
        print(f"✓ PDF: {len(doc)} páginas")
        
        # Convertir a imagen
        cv_image, scale = page_to_image(page)
        print(f"✓ Imagen: {cv_image.shape[1]}x{cv_image.shape[0]} (escala: {scale:.2f})")
        
        # Guardar original
        orig_rgb = cv2.cvtColor(cv_image, cv2.COLOR_BGR2RGB)
        orig_path = os.path.join(DEBUG_OUTPUT_DIR, "layout_original.png")
        Image.fromarray(orig_rgb).save(orig_path)
        print(f"✓ Original: {orig_path}")
        
        # Recortar
        cropped_cv, offset_x, offset_y = crop_margins(cv_image)
        crop_rgb = cv2.cvtColor(cropped_cv, cv2.COLOR_BGR2RGB)
        crop_path = os.path.join(DEBUG_OUTPUT_DIR, "layout_cropped.png")
        Image.fromarray(crop_rgb).save(crop_path)
        print(f"✓ Recortada: {crop_path}")
        
        # Detectar layout
        print(f"\n🎯 DETECCIÓN INTELIGENTE:")
        layout = detector.detect(cropped_cv)
        
        print(f"  ✅ Detectados: {len(layout)} elementos")
        if layout:
            tipos = [element.type for element in layout]
            unique, counts = np.unique(tipos, return_counts=True)
            print(f"      Distribución: {dict(zip(unique, counts))}")
        
        # Crear visualización
        if layout:
            viz_path = os.path.join(DEBUG_OUTPUT_DIR, "layout_smart.png")
            create_visualization(cropped_cv, layout, viz_path)
        
        # Extraer texto
        print(f"\n�� EXTRACCIÓN DE TEXTO:")
        elementos_con_texto = extract_text_from_layout(page, layout, scale, offset_x, offset_y)
        print(f"  ✅ Elementos con texto: {len(elementos_con_texto)}/{len(layout)}")
        
        # Mostrar primeros elementos
        for i, elemento in enumerate(elementos_con_texto[:5], 1):
            preview = elemento.texto[:80] + '...' if len(elemento.texto) > 80 else elemento.texto
            print(f"\n  📄 {i}. {elemento.tipo.upper()} (conf: {elemento.confianza:.2f})")
            print(f"      \"{preview}\"")
        
        # Buscar artículos
        print(f"\n🔍 BÚSQUEDA DE ARTÍCULOS:")
        articulos = []
        for elemento in elementos_con_texto:
            if re.search(r'art[íi]culo\s+\d+', elemento.texto, re.IGNORECASE):
                articulos.append(elemento)
        
        if articulos:
            print(f"  🎯 ¡Encontrados {len(articulos)} artículos!")
            for i, art in enumerate(articulos[:3], 1):
                preview = art.texto[:60] + '...' if len(art.texto) > 60 else art.texto
                print(f"     {i}. [{art.tipo}] \"{preview}\"")
        else:
            print(f"  ℹ️ No se encontraron artículos")
        
        doc.close()
        
        # Resumen
        print(f"\n🎉 RESUMEN:")
        print(f"  📊 Elementos detectados: {len(layout)}")
        print(f"  📝 Con texto válido: {len(elementos_con_texto)}")
        print(f"  ⚖️ Artículos encontrados: {len(articulos)}")
        print(f"\n📁 VER RESULTADOS:")
        print(f"  → layout_smart_detailed.png (¡IMAGEN PRINCIPAL!)")
        
        return layout, elementos_con_texto
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return lp.Layout(), []

print("✅ Función de análisis lista")

✅ Función de análisis lista


In [11]:
# EJECUCIÓN AUTOMÁTICA
print("🚀 INICIANDO ANÁLISIS LAYOUTPARSER INTELIGENTE")
print("=" * 60)

# Buscar PDFs
project_root = "/Users/alexa/Projects/cdmx_kg"
search_paths = [
    os.path.join(project_root, "Mexico_City", "laws"),
    os.path.join(project_root, "Mexico_City", "laws_1"),
    project_root
]

pdf_files = []
for search_path in search_paths:
    if os.path.exists(search_path):
        for root, dirs, files in os.walk(search_path):
            for file in files:
                if file.endswith('.pdf'):
                    pdf_files.append(os.path.join(root, file))

if pdf_files:
    pdf_path = pdf_files[0]
    print(f"📄 Analizando: {os.path.basename(pdf_path)}")
    print(f"📍 Ruta: {pdf_path}")
    print()
    
    # Ejecutar análisis
    layout, elementos_con_texto = analyze_pdf_simple(pdf_path)
    
    print(f"\n" + "="*60)
    print(f"🎯 RESULTADO FINAL:")
    if layout and len(layout) > 0:
        print(f"  ✅ LayoutParser detectó {len(layout)} elementos")
        print(f"  📝 {len(elementos_con_texto)} con texto válido")
        print(f"  🎨 Visualización profesional generada")
        print(f"  🔍 Abrir: {DEBUG_OUTPUT_DIR}/layout_smart_detailed.png")
        
        print(f"\n💡 VENTAJAS LOGRADAS:")
        print(f"  ✅ Clasificación automática por tipo")
        print(f"  ✅ Visualización con colores y scores")
        print(f"  ✅ Filtrado inteligente de duplicados")
        print(f"  ✅ Compatible con layoutparser estándar")
    else:
        print(f"  ❌ No se detectaron elementos")
        print(f"  💡 Revisar configuración de recorte")
        
else:
    print("❌ No se encontraron PDFs en el proyecto")

🚀 INICIANDO ANÁLISIS LAYOUTPARSER INTELIGENTE
📄 Analizando: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
📍 Ruta: /Users/alexa/Projects/cdmx_kg/pdfs/LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf

🎯 ANÁLISIS LAYOUTPARSER: LEY_DE_EDUCACION_DE_LA_CDMX_3.4.pdf
✓ PDF: 25 páginas
✓ Imagen: 1700x2200 (escala: 2.78)
✓ Original: ../data/debug/layout_original.png
📐 Recorte: 1700x2200 → 1600x1910
✓ Recortada: ../data/debug/layout_cropped.png

🎯 DETECCIÓN INTELIGENTE:
    Threshold 150: 1792 contornos
    Threshold 180: 1781 contornos
    Threshold 200: 1778 contornos
    Threshold 220: 1772 contornos
    Threshold 240: 1764 contornos
    📊 Total contornos encontrados: 8887
    🔍 Rechazados por tamaño: 8769
    🔍 Rechazados por duplicado: 83
    ✅ Bloques válidos procesados: 35
  ✅ Detectados: 35 elementos
      Distribución: {'list': 35}
  💾 Básica: ../data/debug/layout_smart_basic.png
  💾 Detallada: ../data/debug/layout_smart_detailed.png

�� EXTRACCIÓN DE TEXTO:
  ✅ Elementos con texto: 0/35

🔍 BÚSQUEDA DE ARTÍCUL